In [1]:
'''
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install sklearn
!pip install dmba
!pip install statsmodels
'''

'\n!pip install numpy\n!pip install pandas\n!pip install matplotlib\n!pip install sklearn\n!pip install dmba\n!pip install statsmodels\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

from dmba import stepwise_selection
from dmba import AIC_score

from dmba import backward_elimination



%matplotlib inline

no display found. Using non-interactive Agg backend


# Data Preparation

We're working with a lot of categorical and boolean variables, we need to convert them to numeric so that our regression functions can properly injest them

In [16]:
df = pd.read_csv('data/medical_clean.csv')

predictors = ['Age', 'Gender', 'HighBlood', 'Stroke', 'Complication_risk',
             'Overweight', 'Arthritis', 'Diabetes', 'Hyperlipidemia', 'BackPain',
             'Anxiety', 'Allergic_rhinitis', 'Reflux_esophagitis', 'Asthma', 
             'VitD_levels', 'Soft_drink', 'Services',
             'vitD_supp', 'Initial_admin']



df['Gender'] = pd.Categorical(df['Gender'])
df['Gender'] = df['Gender'].cat.codes

df['HighBlood'] = pd.Categorical(df['HighBlood'])
df['HighBlood'] = df['HighBlood'].cat.codes

df['Complication_risk'] = pd.Categorical(df['Complication_risk'])
df['Complication_risk'] = df['Complication_risk'].cat.codes

df['Stroke'] = pd.Categorical(df['Stroke'])
df['Stroke'] = df['Stroke'].cat.codes

df['Overweight'] = pd.Categorical(df['Overweight'])
df['Overweight'] = df['Overweight'].cat.codes

df['Arthritis'] = pd.Categorical(df['Arthritis'])
df['Arthritis'] = df['Arthritis'].cat.codes

df['Hyperlipidemia'] = pd.Categorical(df['Hyperlipidemia'])
df['Hyperlipidemia'] = df['Hyperlipidemia'].cat.codes

df['BackPain'] = pd.Categorical(df['BackPain'])
df['BackPain'] = df['BackPain'].cat.codes

df['Anxiety'] = pd.Categorical(df['Anxiety'])
df['Anxiety'] = df['Anxiety'].cat.codes

df['Allergic_rhinitis'] = pd.Categorical(df['Allergic_rhinitis'])
df['Allergic_rhinitis'] = df['Allergic_rhinitis'].cat.codes

df['Reflux_esophagitis'] = pd.Categorical(df['Reflux_esophagitis'])
df['Reflux_esophagitis'] = df['Reflux_esophagitis'].cat.codes

df['Asthma'] = pd.Categorical(df['Asthma'])
df['Asthma'] = df['Asthma'].cat.codes

df['Soft_drink'] = pd.Categorical(df['Soft_drink'])
df['Soft_drink'] = df['Soft_drink'].cat.codes

df['Services'] = pd.Categorical(df['Services'])
df['Services'] = df['Services'].cat.codes

df['Initial_admin'] = pd.Categorical(df['Initial_admin'])
df['Initial_admin'] = df['Initial_admin'].cat.codes

df['Diabetes'] = pd.Categorical(df['Diabetes'])
df['Diabetes'] = df['Diabetes'].cat.codes

outcome = 'Initial_days'
df[outcome] = pd.Categorical(df[outcome])
df[outcome] = df[outcome].cat.codes


X = df[predictors]
y = df[outcome].values

X.head()

,Age,Gender,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,VitD_levels,Soft_drink,Services,vitD_supp,Initial_admin
0,53,1,1,0,2,0,1,1,0,1,1,1,0,1,19.141466,0,0,0,1
1,51,0,1,0,0,1,0,0,0,0,0,0,1,0,18.940352,0,2,1,1
2,53,0,1,0,2,1,0,1,0,0,0,0,0,0,18.057507,0,0,0,0
3,78,1,0,1,2,0,1,0,0,0,0,0,1,1,16.576858,0,0,0,0
4,22,0,0,0,1,0,0,0,1,0,0,1,0,0,17.439069,1,1,2,0


In [17]:
# outcome = 'ReAdmis'
# df[outcome] = pd.Categorical(df[outcome])
# df[outcome] = df[outcome].cat.codes
# df[outcome].head()

# df.ReAdmis = pd.Categorical(df.ReAdmis)
# df.ReAdmis = df.ReAdmis.cat.codes

# Initial Regression

In [18]:
model = LinearRegression()
model.fit(X, y)

#model.summary()
# print(f'Intercept: {model.intercept_:.3f}')
# print(f'Coefficients:')
# for name, coef in zip(predictors, model.coef_):
#     print(f' {name}: {coef}')

LinearRegression()

## Assessing the model's accuracy

In [19]:
from sklearn.metrics import r2_score

fitted = model.predict(X)
r2 = r2_score(y, fitted)
print(f'r2: {r2:.4f}')

r2: 0.0018


In [20]:

model = linear_model.LinearRegression(normalize=False, fit_intercept=True)

def r2_est(X, y):
    model.fit(X, y)
    fitted = model.predict(X)
    return r2_score(y, fitted)

print ('Baseline R2: %0.3f' %  r2_est(X,y))

r2_impact = list()
for i in range(len(X.columns)):
    selection = [j for j in range(len(X.columns)) if i != j]
    r2_impact.append(((r2_est(X, y) - r2_est(X.values[:,selection], y)), X.columns[i]))
    
for imp, varname in sorted(r2_impact, reverse=True):
    print('%6.3f %s' % (imp, varname))

Baseline R2: 0.002
 0.000 Arthritis
 0.000 BackPain
 0.000 Age
 0.000 vitD_supp
 0.000 Reflux_esophagitis
 0.000 Anxiety
 0.000 HighBlood
 0.000 Asthma
 0.000 Services
 0.000 Overweight
 0.000 Soft_drink
 0.000 Gender
 0.000 Initial_admin
 0.000 Stroke
 0.000 Allergic_rhinitis
 0.000 VitD_levels
 0.000 Complication_risk
 0.000 Diabetes
 0.000 Hyperlipidemia


In [21]:
# Code based on page Chapter 4: "Regression and Prediction" from "Practical Statistics for Data Scientists"


#X = pd.get_dummies(df[predictors], drop_first=True)
#y = df[outcome]

In [22]:
'''
from dmba import forward_selection

X = pd.get_dummies(df[predictors], drop_first=True)
y = df[outcome]

def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(X[variables], y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(y, [y.mean()] * len(y), model, df=1)
    return AIC_score(y, model.predict(X[variables]), model)


def score_model(model, variables):
    if len(variables) == 0:
        return r2_score(y, [y.mean()] * len(y))
    fitted = model.predict(X[variables])
    return r2_score(y, fitted)

best_model, best_variables = backward_elimination(X.columns, train_model, score_model, verbose=True)



reduced_predictors = []

print()
print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    reduced_predictors.append(name)
    print(f' {name}: {coef}')
'''

"\nfrom dmba import forward_selection\n\nX = pd.get_dummies(df[predictors], drop_first=True)\ny = df[outcome]\n\ndef train_model(variables):\n    if len(variables) == 0:\n        return None\n    model = LinearRegression()\n    model.fit(X[variables], y)\n    return model\n\ndef score_model(model, variables):\n    if len(variables) == 0:\n        return AIC_score(y, [y.mean()] * len(y), model, df=1)\n    return AIC_score(y, model.predict(X[variables]), model)\n\n\ndef score_model(model, variables):\n    if len(variables) == 0:\n        return r2_score(y, [y.mean()] * len(y))\n    fitted = model.predict(X[variables])\n    return r2_score(y, fitted)\n\nbest_model, best_variables = backward_elimination(X.columns, train_model, score_model, verbose=True)\n\n\n\nreduced_predictors = []\n\nprint()\nprint(f'Intercept: {best_model.intercept_:.3f}')\nprint('Coefficients:')\nfor name, coef in zip(best_variables, best_model.coef_):\n    reduced_predictors.append(name)\n    print(f' {name}: {coef}'

Now we rebuild the model with the reduced list of predictors

In [23]:
'''
model = LinearRegression()
model.fit(df[reduced_predictors], df[outcome])

print(f'Intercept: {model.intercept_:.3f}')
print(f'Coefficients:')
for name, coef in zip(reduced_predictors, model.coef_):
    print(f' {name}: {coef}')
    '''

"\nmodel = LinearRegression()\nmodel.fit(df[reduced_predictors], df[outcome])\n\nprint(f'Intercept: {model.intercept_:.3f}')\nprint(f'Coefficients:')\nfor name, coef in zip(reduced_predictors, model.coef_):\n    print(f' {name}: {coef}')\n    "

In [24]:

'''fitted = model.predict(df[reduced_predictors])
r2 = r2_score(df[outcome], fitted)
print(f'r2: {r2:.4f}')

#aic = AIC_score(y, fitted)'''

"fitted = model.predict(df[reduced_predictors])\nr2 = r2_score(df[outcome], fitted)\nprint(f'r2: {r2:.4f}')\n\n#aic = AIC_score(y, fitted)"

In [25]:

#adapted from D208 lesson 3.3
X = df[predictors]
y = df[outcome]
del X['Initial_days']
predictors.remove('Initial_days')

model = linear_model.LinearRegression(normalize=False, fit_intercept=True)

def r2_est(X, y):
    model.fit(X, y)
    fitted = model.predict(X)
    return r2_score(y, fitted)

print ('Baseline R2: %0.3f' %  r2_est(X,y))

KeyError: 'Initial_days'

In [26]:


r2_impact = list()
for i in range(len(predictors)):
    selection = [j for j in range(len(predictors)) if i != j]
    r2_impact.append(((r2_est(X, y) - r2_est(X.values[:,selection], y)), predictors[i]))
    
for imp, varname in sorted(r2_impact, reverse=True):
    print('%6.3f %s' % (imp, varname))

 0.000 Arthritis
 0.000 BackPain
 0.000 Age
 0.000 vitD_supp
 0.000 Reflux_esophagitis
 0.000 Anxiety
 0.000 HighBlood
 0.000 Asthma
 0.000 Services
 0.000 Overweight
 0.000 Soft_drink
 0.000 Gender
 0.000 Initial_admin
 0.000 Stroke
 0.000 Allergic_rhinitis
 0.000 VitD_levels
 0.000 Complication_risk
 0.000 Diabetes
 0.000 Hyperlipidemia
